**TURN OFF AQE**

In [0]:
spark.conf.set("spark.sql.adaptive.enabled", "false")

In [0]:
# Checking AQE Status
spark.conf.get("spark.sql.adaptive.enabled")

In [0]:
from pyspark.sql.functions import *

**Data Reading**

In [0]:
df = spark.read.format("csv")\
        .option("inferSchema",True)\
        .option("header",True)\
        .load("/FileStore/rawdata/BigMart_Sales.csv")

In [0]:
display(df)

**Get No. of Partitions**

In [0]:
df.rdd.getNumPartitions()


**Changing DEFAULT Partition Size to 128KB**

In [0]:
# Changing the default partition size to 128KB 

spark.conf.set("spark.sql.files.maxPartitionBytes", 131072)

In [0]:
df.rdd.getNumPartitions()

**Changing the default partition size to 128MB**

In [0]:
spark.conf.set("spark.sql.files.maxPartitionBytes", 134217728)

**Repartitioning**

In [0]:
df = df.repartition(10)

In [0]:
df.rdd.getNumPartitions()

**Get Partition Info**

In [0]:
# Function to get the partition id

df = df.withColumn("partition_id",spark_partition_id())

df.display()

**Data Writing**

In [0]:
df.write.format("parquet")\
    .mode("append")\
    .option("path","/FileStore/rawdata/parquetWrite")\
    .save()

**New Data Reading**

In [0]:
df_new = spark.read.format("parquet")\
              .load("/FileStore/rawdata/parquetWrite")

df_new = df_new.filter(col("Outlet_Location_Type") == 'Tier 1')

In [0]:
df_new.display()

**SCANNING OPTIMIZATION**

In [0]:
df.write.format("parquet")\
      .mode("append")\
      .partitionBy("Outlet_Location_Type")\
      .option("path","/FileStore/rawdata/parquetWriteOpt")\
      .save()

In [0]:
df_new = spark.read.format("parquet")\
              .load("/FileStore/rawdata/parquetWriteOpt")

df_new = df_new.filter(col("Outlet_Location_Type") == 'Tier 1')

df_new.display()